In [2]:
import mysql.connector
import openai
import time
import pandas as pd
from tqdm import tqdm



In [3]:
# 建立 MySQL 連線
def create_connection():
    try:
        connection = mysql.connector.connect(
            host='127.0.0.1',
            database='kol_1017',
            user='root',
            password='123456789',
            port=3306
        )
        if connection.is_connected():
            print("✅ MySQL 連線成功")
            return connection
    except mysql.connector.Error as e:
        print(f"❌ 連線錯誤: {e}")
    return None

# 測試 MySQL 連線
conn = create_connection()
if conn:
    conn.close()


✅ MySQL 連線成功


In [4]:
# 讀取 MySQL 資料，從 id=1 開始，按照 ID 遞增排序
def fetch_articles():
    connection = create_connection()
    if connection is None:
        return []
    
    query = "SELECT id, article FROM kol_post_detail WHERE id >= 1 AND article IS NOT NULL ORDER BY id ASC"
    cursor = connection.cursor(dictionary=True)
    cursor.execute(query)
    articles = cursor.fetchall()
    # **去除換行符號 (\n)**
    for article in articles:
        article["article"] = article["article"].replace("\n", " ").replace("\r", " ")  # 去掉所有換行符號
    cursor.close()
    connection.close()
    
    print(f"✅ 成功讀取 {len(articles)} 筆貼文（從 id=1 開始）")
    return articles  # 回傳包含 id 和 article 的列表

# 測試讀取資料（檢查前 5 筆）
articles = fetch_articles()
articles[:5]  # 顯示前 5 筆


✅ MySQL 連線成功
✅ 成功讀取 24419 筆貼文（從 id=1 開始）


[{'id': 1,
  'article': 'Happy birthday 🎂  聽說生日願望許別人的願望才會實現 那我的願望就交給你們來許吧  謝謝很久以前媽媽在這天帶我來這個世界 媽咪母難日快樂  #birthday #happybirthday'},
 {'id': 2,
  'article': '我的課程真的來囉！ 在12/16號之前報名還有禮物唷！  最先要推出的是《丟球運動》課程 一直以來都很想跟大家介紹這個運動 任何人無論男女老少都能玩 而且最不容易受傷同時又有趣的運動  所有詳細內容以及介紹都在這裡 👉🏻@foreveryoung_sports 以後所有運動相關的消息、 教學的內容都會在這個IG帳號 點選這個IG 主頁加入我的教學官方Line 期待我們課堂上見唷！  #吳承洋#吳承洋chriswu #丟球#運動#丟球運動'},
 {'id': 3,
  'article': '終於來看這個愛了一輩子的偶像 從小學就每一首歌都會唱 連歌詞都不用背 我以前帶周杰倫CD去學校放 就是班上最酷的仔 他的歌陪伴了我多少的青春歲月 現在歌單裡都還是有周杰倫組曲 每一次聽都由衷讚嘆他創作能力 沒想到已經多少年過去了 少年都變大叔了 現在依舊很好聽很感動 終於看到現場表演 人生其中一項願望實現了 @jaychou  原本以為只能在地下道聽了 結果居然有生日禮物 實在是很感謝！ 太開心了❤️'},
 {'id': 4,
  'article': '其實人一天能吸收的資訊有限 過度的被餵太多訊息反而會造成不必要的壓力 我常會把推薦內容關掉 避免自己不小心吸收過多資訊 很多時候停下來看看身邊的景色 放緩腳步才能使思緒更加清晰 最近的天色都好美 大家有注意到嗎？  教課內容資訊快籌備好了 Coming soon ~'},
 {'id': 5,
  'article': '男生穿搭 細節是最重要的 要有高級感的打扮 對我來說 肯定是減法 越簡單越需要有質感的襯托 這時選的配件就尤其重要了 太陽眼鏡一直是我非常喜愛的搭配單品 而手機殼、錶殼 則是我最近發現可以大加分的配件 Golden Concept 的手機殼、錶殼 正好滿足了我喜歡細節質感低調內斂 為一體的需求 當然 想要徹底展現自己潮流品味的人士 也可以在這裡找到閃亮眾人的商品  你也是追求高質感配件的人嗎？ Gol

使用openai進行分析

In [ ]:
# 設定 OpenAI API 金鑰
client = openai.Client(api_key="")

In [6]:


def classify_post_with_topic(text, max_retries=3):
    retries = 0  # 計數重試次數

    while retries < max_retries:
        try:
            # 呼叫 OpenAI API 進行貼文分類與主題分析
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": 
                     "你是一位專業的社群媒體內容分析師，負責判斷社群貼文的業配類型，並分析其主要主題。\n"
                    "請根據以下七種類別來分類貼文：\n"
                    "0. 非業配貼文（純個人分享，無商業或推廣性質）\n"
                    "1. 廣告業配（品牌贊助 / 產品推廣）（感謝品牌 / 強調產品特色）\n"
                    "2. 置入式業配（隱性業配）（未明確說明是業配，但有@產品的官方帳號，例如 @gucci，或是加入生活分享）\n"
                    "3. 自我宣傳（個人品牌 / 內容導流）（個人品牌(產品)或自媒體內容自行推廣，無外部合作，粉絲觀看 KOL 的影片、Podcast、課程）\n"
                    "4. 活動邀請 / 體驗業配 (推廣品牌活動 / 體驗，或是店家、展覽、餐廳...等營業場所邀約）\n"
                    "5. 公益/NGO（與商業無關，純粹為社會公益目的）\n"
                    "6. 聯盟行銷（這類貼文的目標是讓粉絲使用折扣碼 / 連結購買產品，經常包含導購連結與限時優惠）\n\n"
                    "此外，如果該貼文是業配（1-6 類），請分析其業配主題，也可以透過官方帳號來分辨業配主題\n"
                    "請從以下 21 種可能的業配主題中，選擇最接近的主題名稱：\n"
                    "1. **3C科技**（手機、筆電、耳機、智慧家電等）\n"  
                    "2. **美妝保養**（彩妝、護膚、醫美）\n"  
                    "3. **服飾配件**（服裝、鞋包、飾品、潮流穿搭）\n"  
                    "4. **運動健身**（健身器材、運動服、健身課程、戶外活動）\n"  
                    "5. **旅遊住宿**（飯店、民宿、景點、交通）\n"  
                    "6. **美食餐飲**（餐廳、速食、甜點、飲料、食品）\n"  
                    "7. **健康保健**（保健食品、營養補充品、醫療健康）\n"  
                    "8. **家居生活**（家具、家電、裝潢、收納）\n"  
                    "9. **財經理財**（投資、保險、信用卡、理財課程）\n"  
                    "10. **教育學習**（線上課程、學習平台、語言學習、職涯發展）\n"  
                    "11. **親子育兒**（嬰幼兒用品、家庭活動、育兒分享）\n"  
                    "12. **遊戲動漫**（電玩、手遊、桌遊、動漫週邊）\n"  
                    "13. **影視娛樂**（電影、戲劇、音樂、演唱會）\n"  
                    "14. **寵物用品**（飼料、玩具、寵物照顧）\n"  
                    "15. **汽車機車**（新車、改裝、租車、共享交通）\n"  
                    "16. **文創設計**（插畫、手作、藝術、設計品牌）\n"  
                    "17. **社群工具**（APP 推薦、社群平台、數位行銷）\n"  
                    "18. **公益倡議 & 社會議題（社會運動、環保倡議、弱勢關懷） **）\n"  
                    "19. **工作職場**（求職平台、職場技巧、企業文化）\n"  
                    "20. **奢侈品精品**（珠寶、名牌、收藏品、限量版商品）\n"  
                    "21. **攝影 & 影像（相機、攝影器材、剪輯軟體、攝影教學）** \n"
                    "請輸出格式：[業配類型],[主題]\n"
                    "如果貼文不是業配 (0)，請將主題設為 '無'"
                    "例如：1,美妝保養 或 0,無"
                    },
                    {"role": "user", "content": text}
                ]
            )

            # ✅ 修正解析邏輯，確保正確獲取分類與主題
            response_text = response.choices[0].message.content.strip()
            

            # ✅ 直接使用逗號 `,` 解析
            if "," in response_text:
                category_text, topic_text = response_text.split(",", 1)  # 只切割一次，確保主題不被切斷
                category_text = category_text.strip()
                topic_text = topic_text.strip()

                # ✅ 確保 category 是有效的數字
                if category_text.isdigit() and 0 <= int(category_text) <= 6:
                    category = int(category_text)
                else:
                    raise ValueError(f"⚠️ 無效的業配類型: {category_text}")

                return category, topic_text

            else:
                raise ValueError("⚠️ API 回應格式錯誤: " + str(response_text))

        except (openai.APIError, ValueError) as e:
            #是哪一篇貼文
            print(f"❌ API 或解析錯誤: {e}")
            return "未知", "未知"

        except Exception as e:
            print(f"❌ 其他錯誤: {e}")
            return "未知", "未知"

    print("❌ 連續 3 次 API 失敗，跳過此貼文")
    return "未知", "未知"



In [7]:
# 批次處理貼文並更新 MySQL（每 10 篇批次更新）
def classify_and_update_test():
    articles = fetch_articles()
    if not articles:
        print("❌ 無資料可處理")
        return
    
    test_articles = articles[20500:]
    print(f"📌 測試貼文數量: {len(test_articles)} 篇")

    connection = create_connection()
    cursor = connection.cursor()

    results = []
    
    # **批量分類，減少 API 請求次數**
    for i, post in enumerate(tqdm(test_articles, desc="📊 分類貼文中", unit="篇"), start=1):
        category, topic = classify_post_with_topic(post["article"])
        results.append((category, topic, post["id"]))  # SQL 執行順序: (ad_label, ad_type, id)
        time.sleep(0.1)  # ✅ 避免 API 過載
        
        # **每 10 篇批次更新**
        if i % 100 == 0:
            try:
                sql = "UPDATE kol_post_detail SET ad_label = %s, ad_type = %s WHERE id = %s"
                cursor.executemany(sql, results)  # ✅ 使用 executemany() 批量更新
                connection.commit()
                print(f"✅ 已批量更新 {len(results)} 篇貼文至資料庫")
                results.clear()  # 清空暫存結果
            except Exception as e:
                print(f"❌ 批量更新失敗，錯誤: {e}")

    # **確保最後一批（不足 10 篇）也能更新**
    if results:
        try:
            sql = "UPDATE kol_post_detail SET ad_label = %s, ad_type = %s WHERE id = %s"
            cursor.executemany(sql, results)
            connection.commit()
            print(f"✅ 最後一批 {len(results)} 篇貼文更新至資料庫")
        except Exception as e:
            print(f"❌ 最後一批更新失敗，錯誤: {e}")

    cursor.close()
    connection.close()
    print("✅ 測試完成，所有貼文已更新至資料庫")





In [8]:
# # 執行測試
# classify_and_update_test()

In [9]:

classify_and_update_test()

✅ MySQL 連線成功
✅ 成功讀取 24419 筆貼文（從 id=1 開始）
📌 測試貼文數量: 3919 篇
✅ MySQL 連線成功


📊 分類貼文中:   3%|▎         | 100/3919 [01:09<2:19:04,  2.19s/篇]

✅ 已批量更新 100 篇貼文至資料庫


📊 分類貼文中:   5%|▌         | 200/3919 [02:16<2:06:17,  2.04s/篇]

✅ 已批量更新 100 篇貼文至資料庫


...

✅ 已批量更新 100 篇貼文至資料庫


📊 分類貼文中: 100%|█████████▉| 3912/3919 [48:21<00:05,  1.35篇/s]

❌ API 或解析錯誤: ⚠️ 無效的業配類型: 這則貼文提到了一個品牌的新品上市（洗髮精），並且采取了一種幽默的方式來引起注意，因此被歸類為置入式業配。貼文雖然不是明確的宣傳，但使用了產品名稱和相關的標籤。

所以，分類為：[2


📊 分類貼文中: 100%|██████████| 3919/3919 [48:25<00:00,  1.35篇/s]


✅ 最後一批 19 篇貼文更新至資料庫
✅ 測試完成，所有貼文已更新至資料庫
